## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `Solucao RAG.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [11]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

In [ ]:
# Load dos modelos (Embeddings e LLM)
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=200,
)

In [14]:
# Carregar o PDF

pdf_link = "DOC-SF238339076816-20230503.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()


In [15]:
# Separar em Chunks (Pedaços de documento)

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
)

chunks = text_spliter.split_documents(pages)

In [17]:
# Salvar no Vector DB - Chroma

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
# db.persist()


In [26]:
# Carregar DB
vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

# Load Retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Chain - Contrução da cadeira de prompt para chamada do LLM
prompt = PromptTemplate.from_template(
    "Utilize o seguinte contexto extraído de documentos PDF para responder à pergunta:\n\n"
    "{context}\n\n"
    "Pergunta: {question}\n\n"
    "Resposta:"
)
chain = create_stuff_documents_chain(llm, prompt)

In [32]:
def ask(question):
    # Recupera os documentos mais relevantes (lista de Document)
    context = retriever.get_relevant_documents(question)
    
    # Passa os documentos como 'input_documents' e a pergunta
    result = chain.invoke({
        "context": context,
        "question": question
    })
    
    # Extrai a resposta
    answer = result
    
    return answer, context


In [35]:
user_question = input("User: ")
answer, context = ask(user_question)
print(f"Question: {user_question}")
print("Answer:", answer)


Question: Alguns dos principais pontos da Lei de Inteligência Artificial mencionada são
Answer: - Estabelecimento de direitos para proteção das pessoas impactadas por sistemas de inteligência artificial
- Disposição de ferramentas de governança e supervisão
- Regulação baseada em riscos e modelagem regulatória fundada em direitos
- Definição de fundamentos e princípios gerais para o desenvolvimento e utilização de sistemas de inteligência artificial
- Proteção dos direitos das pessoas afetadas por sistemas de inteligência artificial, incluindo acesso à informação, direito de contestar decisões automatizadas, direito à não-discriminação e correção de vieses discriminatórios
- Intensificação da obrigação de informação e transparência quando o sistema de IA produz efeitos jurídicos relevantes ou impacta os sujeitos de maneira significativa
- Calibração da regulação de acordo com os potenciais r


In [36]:
for c in context:
    print(c)

page_content='liberdades fundamentais, a valorização do trabalho e da dignidade da pessoa 
humana e a inovação tecnológica representada pela inteligência artificial. 
O projeto tem um duplo objetivo. De um lado, estabelece direitos 
para proteção do elo mais vulnerável em questão, a pessoa natural que já é 
diariamente impactada por sistemas de inteligência artificial, desde a 
recomendação de conteúdo e direcionamento de publicidade na Internet até a 
sua análise de elegibilidade para tomada de crédito e para determinadas 
políticas públicas. De outro lado, ao dispor de ferramentas de governança e de 
um arranjo institucional de fiscalização e supervisão, cria condições de 
previsibilidade acerca da sua interpretação e, em última análise, segurança 
jurídica para inovação e o desenvolvimento tecnológico. 
A proposição parte da premissa, portanto, de que não há um trade-
off entre a proteção de direitos e liberdades fundamentais, da valorização do 
trabalho e da dignidade da pessoa hum